In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
path = "./data/News/"

train_emb = np.load(path + "train_embeddings.npy")
train_label = np.load(path + "train_labels.npy")
test_emb = np.load(path + "test_embeddings.npy")
test_label = np.load(path + "test_labels.npy")

In [7]:
def classifier(X_emb, y_emb): 
    similarity = cosine_similarity(X_emb, y_emb)
    preds = np.argmax(similarity, axis=0)
    return preds

def accuracy(preds, labels): 
    return np.mean([preds == labels])